# Read architecture from RSML file and run a simulation
This is the notebook version of the example in read the doc. It is a small example to illustrate how to use the RSML format (http://rootsystemml.github.io/). The architecture is the arabidopsis-simple example http://rootsystemml.github.io/images/examples/arabidopsis-simple.rsml.

Point to the source files if the notebook is run locally, from a git repository clone for example, without hydroroot installation, but only the dependencies installed.

In [1]:
import rsml
from hydroroot import radius
from hydroroot.main import hydroroot_flow
from oawidgets.plantgl import PlantGL
from hydroroot.display import mtg_scene
from hydroroot.hydro_io import import_rsml_to_discrete_mtg, export_mtg_to_rsml
from pathlib import Path
from pprint import pprint
%run read.py

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [2]:
data = Path('data')/'UC1_HIRROS_arabidopsis'
rsa_files = list(data.glob('*.rsml'))
pprint(rsa_files)

[PosixPath('data/UC1_HIRROS_arabidopsis/UC1_230629PN028.rsml'),
 PosixPath('data/UC1_HIRROS_arabidopsis/UC1_230629PN005.rsml'),
 PosixPath('data/UC1_HIRROS_arabidopsis/UC1_230629PN026.rsml'),
 PosixPath('data/UC1_HIRROS_arabidopsis/UC1_230629PN033.rsml'),
 PosixPath('data/UC1_HIRROS_arabidopsis/UC1_230629PN015.rsml'),
 PosixPath('data/UC1_HIRROS_arabidopsis/UC1_230629PN017.rsml'),
 PosixPath('data/UC1_HIRROS_arabidopsis/UC1_230629PN016.rsml'),
 PosixPath('data/UC1_HIRROS_arabidopsis/UC1_230629PN021.rsml'),
 PosixPath('data/UC1_HIRROS_arabidopsis/UC1_230629PN007.rsml'),
 PosixPath('data/UC1_HIRROS_arabidopsis/UC1_230629PN013.rsml')]


Read the RSML file and convert it into a *continuous* MTG. This is a MTG where each root (primary and lateral) is represented by one vertex. The geometry of each root is then stored in g_c.property('geometry').

In [4]:
g = rsml.rsml2mtg(rsa_files[0])

To be used in HydroRoot the MTG has to be converted to a *discrete* form of MTG, i.e. each vertex represent a representative elementary volume of a given length for example $10^{-4}$ m. In HydroRoot the lengths are in meter, therefore we must retrieve the resolution and the unit of the RSML file. 

In [5]:
resolution = g.graph_properties()['metadata']['resolution'] # pixel to unit
unit = g.graph_properties()['metadata']['unit']
print(unit)

pixel(um)


In [6]:
resolution = resolution * 1e-6 # pixel to unit to m

In [7]:
# Split the forest into 5 independent graphs
g1, g2, g3, g4, g5 = gs =  split(g)

Build the discrete MTG

In [8]:
g = import_rsml_to_discrete_mtg(g1, segment_length = 1.0e-4, resolution = resolution)

Calculate some properties needed to simulate a sap flux from the root under overpressure. 

In [9]:
g = radius.ordered_radius(g, 7.0e-5, 0.7) # root radii
g = radius.compute_relative_position(g) # Compute the position of each segment relative to the axis bearing it

Some conductance data versus distance to tip

In [10]:
k_radial_data=([0, 0.2],[30.0,30.0])
K_axial_data=([0, 0.2],[3.0e-7,4.0e-4])

Flux and equivalent conductance calculation, for a root in an external hydroponic medium at 0.4 MPa, its base at 0.1 MPa, and with the conductances set above.

In [11]:
g, keq, jv = hydroroot_flow(g, psi_e = 0.4, psi_base = 0.1, axial_conductivity_data = K_axial_data, radial_conductivity_data = k_radial_data)

In [12]:
print('equivalent root conductance (microL/s/MPa): ',keq, 'sap flux (microL/s): ', jv)

equivalent root conductance (microL/s/MPa):  0.0003716561491631043 sap flux (microL/s):  0.0001114968447489313


Display the local water uptake heatmap in 3D

In [13]:
s = mtg_scene(g, prop_cmap = 'j') # create a scene from the mtg with the property j is the radial flux in ul/s
PlantGL(s) # display the root into the plantgl oawidget

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

Export the MTG to RSML

At this stage (2022-08-22) only the root length and the branching position are used to simulate architecture in hydroponic solution. The exact position in 3D is not stored in the discrete MTG form and so not exported to RMSL.

In [13]:
export_mtg_to_rsml(g, "test.rsml", segment_length = 1.0e-4)

TODO: time-sequence


The resolution of the RSML data is 1.0e-4 and the unit is meter.